In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 15.6 MB/s 
     |████████████████████████████████| 3.3 MB 77.2 MB/s 
     |████████████████████████████████| 880 kB 83.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=3a66eccfbdfb0ce470bec6cc9ab54f2ee0d7ab2c296c89150d60b96a43fea2fe
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=3c87c2e3edd06a091085c9c6eace95b56f9e107a11bce656e059b4744a35edf3
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
Looking in indexes: https://pypi

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import tensorflow, sentence_transformers, transformers
tensorflow.__version__, sentence_transformers.__version__, transformers.__version__

('2.9.2', '0.4.1', '4.3.3')

In [5]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [6]:
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from src.encoders.context_encoder_ldabert_2 import ContextEncoderComplex, ContextEncoderConv
from src.dataset.ldabert_2 import LDABERT2Dataset

from tensorflow.python import keras
import toml
import json
import pandas as pd
import numpy as np
from utils.experiments import get_experiments_json, get_experiments, save_results

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

## Experiment

In [7]:
experiments_config = get_experiments_json('ldabert2_complex_finetune_low_epochs_test')
experiments_config_df = pd.DataFrame.from_dict(experiments_config)
experiments_config_df

,bert_type,dataset_type,final_dropout,dense_neurons,max_sentence_length,lstm_size,lstm_dropout_percentage,cnn_filters,cnn_kernel_size,pool_size,gamma,pct_data,augment_pct,bert_trainable,epochs
0,ldabert,clinical,0.25,256,128,256,0.25,8,3,2,15,1,1,True,10
1,ldabert,wiki,0.25,256,128,256,0.25,8,3,2,15,1,1,True,10
2,ldabert,fiction,0.25,256,128,256,0.25,8,3,2,15,1,1,True,10


In [8]:
experiments_config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [9]:
def get_random_hash(k):
  import random, string
  x = ''.join(random.choices(string.ascii_letters + string.digits, k=k))
  return x

In [ ]:
for experiment in experiments_config[0:3]: # all
    dataset_type = experiment['dataset_type']
    final_dropout = experiment['final_dropout']
    dense_neurons = experiment['dense_neurons']
    lstm_size = experiment['lstm_size']
    lstm_dropout_percentage = experiment['lstm_dropout_percentage']
    cnn_filters = experiment['cnn_filters']
    cnn_kernel_size = experiment['cnn_kernel_size']
    pool_size = experiment['pool_size']
    
    pct_data = experiment['pct_data']
    augment_pct = experiment['augment_pct']
    gamma = experiment['gamma']
    max_sentence_length = experiment['max_sentence_length']
    bert_trainable = experiment['bert_trainable']
    finetuning_epochs = 5 # finetune the system for 5 epochs before training with the frozen finetuned BERT weights
    epochs = experiment['epochs']
    BATCH_SIZE = 64
    print("params:", experiment)
    random_hash = get_random_hash(5)
    experiment['epochs'] = epochs

    for step in ["finetune","frozen"]:
    # for step in ["frozen"]:
    
      # init model
      print("initializing model...")
      model = ContextEncoderComplex(final_dropout=final_dropout,
                              dense_neurons=dense_neurons,
                                    lstm_size=lstm_size,
                              lstm_dropout_percentage=lstm_dropout_percentage,
                              cnn_filters=cnn_filters,
                              cnn_kernel_size=cnn_kernel_size,
                              pool_size=pool_size,
                            gamma=gamma,
                            max_sentence_length=max_sentence_length,
                            bert_trainable=True if step == "finetune" else False)
      
      # print("number of params: ", sum([np.prod(keras.get_value(w).shape) for w in model.trainable_weights]))
      
      # init dataset
      print("initializing dataset...")
      dataset = LDABERT2Dataset(dataset_type=dataset_type,
                              pct_data=pct_data,
                            max_seq_length=max_sentence_length,
                              max_segment_length=5,
                              augment_pct=augment_pct,
                              split="train",
                        artificial_segments=True)
      
      # process dataset
      print("processing dataset...")
      sentences, tokenized_sentences, labels = dataset.process()

      vectors_filename = '{}_{}_as-{}_msl-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments, dataset.max_segment_length)

      saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

      if len(saved_vectors) == 0:
          saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

      left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
      lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

      # get class weight
      neg, pos = np.bincount(labels.flatten())
      initial_bias = np.log([pos/neg])
      
      total=len(labels)
      weight_for_0 = (1 / neg)*(total)/2.0 
      weight_for_1 = (1 / pos)*(total)/2.0

      class_weight = {0: weight_for_0, 1: weight_for_1}
      print("class weight", class_weight)
      
      # create checkpoint path
      checkpoint_filepath = '{}/models/LDABERT2/hybrid-conv/{}-{}-{}-pct-{}-aug_{}/finetune-{}/checkpoint'.format(
                              config.root_path,
                              dataset.dataset_type,                    
                              len(saved_sentences), 
                              dataset.pct_data,
                              dataset.augment_pct,
                              random_hash,
                              bert_trainable)
      
      # continue training
      if step == "frozen":
        # random_hash = 'WLZ0q'
        checkpoint_filepath = '{}/models/LDABERT2/hybrid-conv/{}-{}-{}-pct-{}-aug_{}/finetune-{}/checkpoint'.format(
                              config.root_path,
                              dataset.dataset_type,                    
                              len(saved_sentences), 
                              dataset.pct_data,
                              dataset.augment_pct,
                              random_hash,
                              bert_trainable)
      
      print(checkpoint_filepath)
      
      # get callbacks ready.
      model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
          filepath=checkpoint_filepath,
          save_weights_only=False,
          monitor='val_accuracy',
          save_best_only=True,
          mode="auto",
          save_freq="epoch")

      early_stopping = tf.keras.callbacks.EarlyStopping(
          monitor='val_accuracy', 
          verbose=1,
          patience=10,
          mode='max',
          restore_best_weights=True)
      
      log_path = '{}/models/LDABERT2/hybrid-conv/{}-{}-{}-pct-{}-aug_{}/finetune-{}/checkpoint/training.log'.format(
                              config.root_path,
                              dataset.dataset_type,                    
                              len(saved_sentences), 
                              dataset.pct_data,
                              dataset.augment_pct,
                              random_hash,
                              bert_trainable)

      csv_logger = tf.keras.callbacks.CSVLogger(log_path, separator=",", append=False)

      callbacks = [
      #     early_stopping,
          model_checkpoint_callback,
          csv_logger
      ]
      
      # compiling model
      print("compiling the model...")
      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                    loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[
                        keras.metrics.BinaryAccuracy(name='accuracy')
                    ])
      
      try:
          model.load_weights(checkpoint_filepath)
          print("model loaded.")
      except:
          print("No checkpoint available.")
      
      print("starting the training process...")
      # remove warnings
      tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
      history = model.fit([
                              left_input, mid_input, right_input, 
                              lda_left_input, lda_mid_input, lda_right_input
                          ], 
                          tf.convert_to_tensor(saved_labels), 
                          epochs=finetuning_epochs if step == "finetune" else epochs,
                          validation_split=0.25,
                          batch_size=BATCH_SIZE,
                          verbose=1, 
                          class_weight=class_weight,
                          callbacks=callbacks)

      # assigning history to experiment object for saving.
      experiment["history"] = history.history
      experiment["hash"] = random_hash
      
      print("saving results...")
      save_results(experiment)

params: {'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.25, 'dense_neurons': 256, 'max_sentence_length': 128, 'lstm_size': 256, 'lstm_dropout_percentage': 0.25, 'cnn_filters': 8, 'cnn_kernel_size': 3, 'pool_size': 2, 'gamma': 15, 'pct_data': 1, 'augment_pct': 1, 'bert_trainable': True, 'epochs': 10}
initializing model...


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


initializing dataset...
processing dataset...
artificial segments True
artificial segments True
class weight {0: 0.6484113712374582, 1: 2.184507042253521}
/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/hybrid-conv/clinical-44979-1-pct-1-aug_gZLlY/finetune-True/checkpoint
compiling the model...
No checkpoint available.
starting the training process...
Epoch 1/5
sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'context_encoder_complex/tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 128, 768) dtype=float32>, pooler_output=<tf.Tensor 'context_encoder_complex/tf_bert_model/bert/pooler/dense/Tanh:0' shape=(None, 768) dtype=float32>, hidden_states=None, attentions=None)
sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'context_encoder_complex/tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 128, 768) dtype=float32>, pooler_output=<tf.Tensor 'conte

...

528/528 [==============================] - 216s 409ms/step - loss: 0.6849 - accuracy: 0.7361 - val_loss: 0.6877 - val_accuracy: 0.7371
Epoch 8/10
528/528 [==============================] - 142s 269ms/step - loss: 0.6850 - accuracy: 0.7366 - val_loss: 0.6857 - val_accuracy: 0.7359
Epoch 9/10
527/528 [============================>.] - ETA: 0s - loss: 0.6850 - accuracy: 0.7379sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'context_encoder_complex_1/tf_bert_model_1/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 128, 768) dtype=float32>, pooler_output=<tf.Tensor 'context_encoder_complex_1/tf_bert_model_1/bert/pooler/dense/Tanh:0' shape=(None, 768) dtype=float32>, hidden_states=None, attentions=None)
sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'tf_bert_model_1/StatefulPartitionedCall:0' shape=(None, 128, 768) dtype=float32>, pooler_output=<tf.Tensor 'tf_bert_model_1/StatefulPartitionedCall:1' shape=(None, 768) dtype

528/528 [==============================] - 214s 406ms/step - loss: 0.6850 - accuracy: 0.7379 - val_loss: 0.6874 - val_accuracy: 0.7374
Epoch 10/10
528/528 [==============================] - 142s 269ms/step - loss: 0.6849 - accuracy: 0.7358 - val_loss: 0.6909 - val_accuracy: 0.7337
saving results...
params: {'bert_type': 'ldabert', 'dataset_type': 'wiki', 'final_dropout': 0.25, 'dense_neurons': 256, 'max_sentence_length': 128, 'lstm_size': 256, 'lstm_dropout_percentage': 0.25, 'cnn_filters': 8, 'cnn_kernel_size': 3, 'pool_size': 2, 'gamma': 15, 'pct_data': 1, 'augment_pct': 1, 'bert_trainable': True, 'epochs': 10}
initializing model...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


initializing dataset...
processing dataset...
artificial segments True
artificial segments True
class weight {0: 0.6378117432462183, 1: 2.3140689183021452}
/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/hybrid-conv/wiki-40561-1-pct-1-aug_iRZW8/finetune-True/checkpoint
compiling the model...
No checkpoint available.
starting the training process...
Epoch 1/5
sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'context_encoder_complex_2/tf_bert_model_2/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 128, 768) dtype=float32>, pooler_output=<tf.Tensor 'context_encoder_complex_2/tf_bert_model_2/bert/pooler/dense/Tanh:0' shape=(None, 768) dtype=float32>, hidden_states=None, attentions=None)
sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'context_encoder_complex_2/tf_bert_model_2/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 128, 768) dtype=float32>, pooler_output=<tf.Tens

476/476 [==============================] - 418s 789ms/step - loss: 0.6941 - accuracy: 0.4921 - val_loss: 0.6831 - val_accuracy: 0.5727
Epoch 2/5
476/476 [==============================] - 297s 624ms/step - loss: 0.6937 - accuracy: 0.5677 - val_loss: 0.6876 - val_accuracy: 0.5719
Epoch 3/5
476/476 [==============================] - 297s 624ms/step - loss: 0.6936 - accuracy: 0.5739 - val_loss: 0.6883 - val_accuracy: 0.5693
Epoch 4/5
476/476 [==============================] - 297s 624ms/step - loss: 0.6936 - accuracy: 0.5802 - val_loss: 0.7053 - val_accuracy: 0.5510
Epoch 5/5
476/476 [==============================] - ETA: 0s - loss: 0.6936 - accuracy: 0.5392

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()